# 1.算法描述

（1）获取样本文件，将样本人工分类，并标记

（2）每个类别的样本中文分词

（3）去除样本中垃圾词条

（4）合成特征组，分析计算词频信息

（5）计算先验概率

（6）读取未知样本，中文分词，去垃圾词，形成样本特征值

（7）计算后验概率，得到最大概率所属类别即为文本所属类别

In [ ]:
__author__ = 'LiFeiteng'  
02.# -*- coding: utf-8 -*-  
03.import os  
04.import  jieba  
05.import nltk  
06.  
07.  
08.## 由搜狗语料库 生成数据  
09.folder_path = 'C:\LIFEITENG\SogouC.reduced\\Reduced'  
10.#folder_path = 'C:\LIFEITENG\SogouC.mini\Sample'  
11.folder_list = os.listdir(folder_path)  
12.class_list = [] ##由于乱码等问题 仅以数字[0,1,...]来代表文件分类  
13.nClass = 0  
14.N = 100 #每类文件 最多取 100 个样本 70%train 30%test  
15.train_set = []  
16.test_set = []  
17.all_words = {}  
18.import time  
19.process_times = [] ## 统计处理每个文件的时间  
20.for i in range(len(folder_list)):  
21.    new_folder_path = folder_path + '\\' + folder_list[i]  
22.    files = os.listdir(new_folder_path)  
23.    class_list.append(nClass)  
24.    nClass += 1  
25.    j = 0  
26.    nFile = min([len(files), N])  
27.    for file in files:  
28.        if j > N:  
29.            break  
30.        starttime = time.clock()  
31.  
32.        fobj = open(new_folder_path+'\\'+file, 'r')  
33.        raw = fobj.read()  
34.        word_cut = jieba.cut(raw, cut_all=False)  
35.        word_list = list(word_cut)  
36.        for word in word_list:  
37.            if word in all_words.keys():  
38.                all_words[word] += 1  
39.            else:  
40.                all_words[word] = 0  
41.        if j > 0.3 * nFile:  
42.            train_set.append((word_list, class_list[i]))  
43.        else:  
44.            test_set.append((word_list, class_list[i]))  
45.        j += 1  
46.        endtime = time.clock()  
47.        process_times.append(endtime-starttime)  
48.  
49.        print "Folder ",i,"-file-",j, "all_words length = ", len(all_words.keys()),\  
50.            "process time:",(endtime-starttime)  
51.  
52.  
53.print len(all_words)  
54.  
55.## 根据word的词频排序  
56.all_words_list = sorted(all_words.items(), key=lambda e:e[1], reverse=True)  
57.word_features = []  
58.## 由于乱码的问题，没有正确使用 stopwords；简单去掉 前100个高频项  
59.## word_features 是选用的 word-词典  
60.for t in range(100, 1100, 1):  
61.    word_features.append(all_words_list[t][0])  
62.  
63.def document_features(document):  
64.    document_words = set(document)  
65.    features = {}  
66.    for word in word_features: ## 根据词典生成 每个document的feature True or False  
67.        features['contains(%s)' % word] = (word in document_words)  
68.    return features  
69.  
70.## 根据每个document 分词生成的 word_list 生成 feature  
71.train_data = [(document_features(d), c) for (d,c) in train_set]  
72.test_data = [(document_features(d), c) for (d,c) in test_set]  
73.print "train number:",len(train_data),"\n test number:",len(test_data)  
74.  
75.## 朴素贝叶斯分类器  
76.classifier = nltk.NaiveBayesClassifier.train(train_data)  
77.print "test accuracy:",nltk.classify.accuracy(classifier, test_data)  
78.  
79.## 处理每个文件所用的时间 可见到后面 处理单个文件的时间显著增长  
80.## 原因 已查明  
81.import pylab  
82.pylab.plot(range(len(process_times)), process_times, 'b.')  
83.pylab.show()  
